In [1]:
userInput = {'x': ['hate', 'love'], 'y': ['violence','peace'], 'z': ['sad','happy'], 'text': 'this is a string of text about happiness and sadness but mostly sadness in fact maybe even despair.  Despair is dark, dark like fighting and war and unquiet nights.'}


In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from gensim.models import word2vec

/usr/local/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [3]:
news = word2vec.Word2Vec.load_word2vec_format('bigFiles/GoogleNews-vectors-negative300.bin', binary=True)


In [83]:
def text_to_words(textfield, model):
    textblock = BeautifulSoup(textfield, 'html.parser').get_text()
    textblock = re.sub("[^a-zA-Z]", " ", textblock)
    words = textblock.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if w in model and not w in stops]
    return words


In [89]:
def project3D(model, xmax, ymax, zmax, words, words2 = None):
    print "WORDS "
    print words
    print "WORDS2"
    print words2
    wordCoordinates={}
    wordCoordinates2={}

    for word in words:
        coords = []
        coords.append(np.asscalar(np.linalg.norm(np.subtract(model[word], xmax))))
        coords.append(np.asscalar(np.linalg.norm(np.subtract(model[word], ymax))))
        coords.append(np.asscalar(np.linalg.norm(np.subtract(model[word], zmax))))
        #coords.append(np.asscalar(np.dot(model[word], xmax)/np.linalg.norm(xmax)))
        #coords.append(np.asscalar(np.dot(model[word], ymax)/np.linalg.norm(ymax)))
        #coords.append(np.asscalar(np.dot(model[word], zmax)/np.linalg.norm(zmax)))
        wordCoordinates[word] = coords

    if words2 != None:
        for word in words2:
            coords = []
            coords.append(np.asscalar(np.dot(model[word], xmax)/np.linalg.norm(xmax)))
            coords.append(np.asscalar(np.dot(model[word], ymax)/np.linalg.norm(ymax)))
            coords.append(np.asscalar(np.dot(model[word], zmax)/np.linalg.norm(zmax)))
            wordCoordinates2[word] = coords

    return {'text1': wordCoordinates, 'text2': wordCoordinates2}


# wordCoordinate = {}
# {'happy' : [.234, .32484387, .984357], 'puppy': [.234234, .4983982, .21425]}

In [90]:
def avgWordVec(arrayOfStrings):
    vecs = [news[w] for w in arrayOfStrings if w in news]
    return reduce(lambda sum, word: sum + word, vecs, np.zeros(news.vector_size))/len(vecs)


In [91]:
def getPoints(userInputObj):
    print userInputObj

    textfield = userInputObj['text']
    wordsToPlot = [text_to_words(textfield, news)]

    if 'text2' in userInputObj:
        print "in text2"
        textfield2 = userInputObj['text2']
        wordsToPlot.append(text_to_words(textfield2, news))

    print wordsToPlot
    xmax = avgWordVec(userInputObj['x'][0])

    ymax = avgWordVec(userInputObj['y'][0])

    zmax = avgWordVec(userInputObj['z'][0])

    words_with_coords = project3D(news, xmax, ymax, zmax, *wordsToPlot)
    return words_with_coords

In [92]:
inputBo =    {
      "x": [["despair", "sorrow", "hopeless"], ["joy", "hope", "happiness"]],
      "y": [["conflict", "strife", "violence"], ["peace", "calm", "tranquility"]],
      "z": [["danger", "warning", "unsafe","harm"], ["security", "safe", "secure"]],
      "text": "My fellow citizens:  I stand here today humbled by the task before us",
      "text2": "Friends, delegates and fellow Americans: I humbly and gratefully accept"
    }



In [63]:
getPoints(inputBo)

{'y': [['conflict', 'strife', 'violence'], ['peace', 'calm', 'tranquility']], 'x': [['despair', 'sorrow', 'hopeless'], ['joy', 'hope', 'happiness']], 'z': [['danger', 'warning', 'unsafe', 'harm'], ['security', 'safe', 'secure']], 'text2': 'Friends, delegates and fellow Americans: I humbly and gratefully accept', 'text': 'My fellow citizens:  I stand here today humbled by the task before us'}
in text2
[[u'fellow', u'citizens', u'stand', u'today', u'humbled', u'task', u'us'], [u'friends', u'delegates', u'fellow', u'americans', u'humbly', u'gratefully', u'accept']]
WORDS 
[u'fellow', u'citizens', u'stand', u'today', u'humbled', u'task', u'us']
WORDS2
[u'friends', u'delegates', u'fellow', u'americans', u'humbly', u'gratefully', u'accept']


{'text1': {u'citizens': [0.4389950429457385,
   0.49476308006783787,
   0.664882929885206],
  u'fellow': [0.2613334917338792, 0.14040720377278554, 0.13166904246603484],
  u'humbled': [0.8129367111675189, 0.4807172799638751, 0.24147339576194976],
  u'stand': [0.1438623356189166, 0.2622651720140307, 0.32395431288080845],
  u'task': [0.408399373039623, 0.2608118734050389, 0.32513731093214626],
  u'today': [0.16049734751631656, 0.11228857618341638, 0.3101258189207309],
  u'us': [0.5784479300288867, 0.37971418810371643, 0.4655427425530187]},
 'text2': {u'accept': [0.37117455107944985,
   0.11694659630447654,
   0.4594992484701202],
  u'americans': [0.24499665542954616,
   -0.06395969554071958,
   0.3075610319214677],
  u'delegates': [0.21149913066157622, 0.3213215667227717, 0.2288671998939325],
  u'fellow': [0.2613334917338792, 0.14040720377278554, 0.13166904246603484],
  u'friends': [0.6966649744286775, 0.3742455942823086, 0.3456705863914378],
  u'gratefully': [0.7628253371499252, 0.505702

In [93]:
getPoints(inputBo)

{'y': [['conflict', 'strife', 'violence'], ['peace', 'calm', 'tranquility']], 'x': [['despair', 'sorrow', 'hopeless'], ['joy', 'hope', 'happiness']], 'z': [['danger', 'warning', 'unsafe', 'harm'], ['security', 'safe', 'secure']], 'text2': 'Friends, delegates and fellow Americans: I humbly and gratefully accept', 'text': 'My fellow citizens:  I stand here today humbled by the task before us'}
in text2
[[u'fellow', u'citizens', u'stand', u'today', u'humbled', u'task', u'us'], [u'friends', u'delegates', u'fellow', u'americans', u'humbly', u'gratefully', u'accept']]
WORDS 
[u'fellow', u'citizens', u'stand', u'today', u'humbled', u'task', u'us']
WORDS2
[u'friends', u'delegates', u'fellow', u'americans', u'humbly', u'gratefully', u'accept']


{'text1': {u'citizens': [3.4299161650167718,
   3.4360548841294336,
   3.1369567600072195],
  u'fellow': [3.017949201439794, 3.399050876270331, 3.049434914392019],
  u'humbled': [3.4655278519151653, 4.12201922398515, 3.942047229878535],
  u'stand': [2.9884470136768, 3.1522549343014554, 2.8139623404725547],
  u'task': [3.52518341969145, 3.6305521685356754, 3.3729691872698657],
  u'today': [2.7376715872758433, 2.988749144089627, 2.5009409484094993],
  u'us': [2.9299983605532964, 3.2188349174290374, 2.822948675254476]},
 'text2': {u'accept': [0.6109574786434685,
   0.2933970677639291,
   0.2509881300697986],
  u'americans': [0.264937737722067, 0.19244216109118722, 0.1644679412899925],
  u'delegates': [0.16456271407778897, 0.3686108708277429, -0.049121867615104],
  u'fellow': [0.38305823119589716, 0.01920908941288398, 0.10358088976059604],
  u'friends': [0.5895224299401265, 0.2349402199556635, 0.24144848663934082],
  u'gratefully': [0.8748425306965303, 0.1378401609675214, 0.101603004078361